### Installing the required Modules


In [1]:
# Importing the require modules
import numpy as np

import os
import pickle 
import blosc

import re
from pprint import pprint

from openbabel import pybel
from openbabel import openbabel as ob
from xtb.interface import Calculator, Param
from xtb.libxtb import VERBOSITY_MUTED

error_files = []

## Helper function filter to extract all the bond lengths, angles and dihedrals from a molecule

In [2]:
def filterAtom(atom: str) -> str:
    '''Removes any number from the string e.g. C3 becomes C'''
    pattern_order = r'[0-9]'
    return re.sub(pattern_order, '', atom)

def atomType(mol, atomIdx) -> str:
    '''get the atomic type given an atom index'''
    return mol.OBMol.GetAtom(atomIdx).GetType()

In [3]:
def getBondOrders_Charges(mol):
    
    coords = []
    for atom in mol.atoms:
        coords.append([atom.coords[0], atom.coords[1], atom.coords[2]])
    
    # xtb-python expects them in Bohr
    atom_coords = np.array(coords) / 0.52917721092
    atomic_nums = np.array([a.atomicnum for a in mol.atoms])
    
    # Creating the calculator
    calc = Calculator(Param.GFN2xTB, atomic_nums, atom_coords)
    calc.set_verbosity(VERBOSITY_MUTED)
    res = calc.singlepoint()
    
    # Get the partial charges
    charges = res.get_charges()
    
    # Get the bond order matrix
    bond_orders = res.get_bond_orders()
    
    '''
    print("Charges:")
    for i, charge in enumerate(charges):
        print("%d\t%f" % (i+1, charge))
        
    print("Bond orders (matrix):")
    for i, row in enumerate(bond_orders):
        for j, bond_order in enumerate(row):
            if j > i and bond_order > 0.1:
                print("%d \t %d \t %f" % (i+1, j+1, bond_order))
    '''
    
    return (charges, bond_orders)

In [4]:
def getBonds(mol, charge, bond_order) -> dict:
    '''Iterate through all the bonds in a molecule'''
    
    bonds = {}
    
    for bond in ob.OBMolBondIter(mol.OBMol):
        index1 = bond.GetBeginAtomIdx()
        index2 = bond.GetEndAtomIdx()
        begin = filterAtom(atomType(mol, index1))
        end = filterAtom(atomType(mol, index2))
        bond_length = round(bond.GetLength(),4)
        bondOrder = bond_order[index1-1][index2-1]
        
        # Swap them for lexographic order
        if (end < begin):
            begin, end = end, begin
        
        #Appending to the dictionary of list
        if (f"{begin} - {end} , {bond_order}" in bonds.keys()):
            bonds[f"{begin} - {end} , {round(bondOrder,1)}"].append((bond_length, charge[index1-1], charge[index2-1], 
                                                             round(bondOrder,2)))
        else:
            bonds[f"{begin} - {end} , {round(bondOrder,1)}"]=[(bond_length, charge[index1-1], charge[index2-1], 
                                                             round(bondOrder,2))]
        
    return bonds

In [22]:
def getAngles(mol, bondOrder, charge) -> dict:
    '''Iterate through all the bond angles in a molecule'''
    
    angles = {}
    
    for angle in ob.OBMolAngleIter(mol.OBMol):
        bond_order1 = bondOrder[angle[0], angle[1]]
        bond_order2 = bondOrder[angle[1], angle[2]]
        #print(bond_order1, bond_order2)
        #print(angle[0], angle[1], angle[2])
        
        a = angle[0] + 1
        b = mol.OBMol.GetAtom(angle[1] + 1)
        c = angle[2] + 1
        bond_angle = round(b.GetAngle(a, c), 3)

        aType = filterAtom(atomType(mol, a))
        bType = filterAtom(b.GetType())
        cType = filterAtom(atomType(mol, c))
        
        # Swap them for lexographic order
        if (cType < aType):
            aType, cType = cType, aType
            bond_order1, bond_order2 = bond_order2, bond_order1
            
        #Appedning to the dictonary of lists if the angle b/t the elements already existse
        if(f"{aType} - {bType} - {cType}" in angles.keys()):
            angles[f"{aType} - {bType} - {cType}"].append((bond_angle, charge[angle[0]], charge[angle[1]], charge[angle[2]], bond_order1, bond_order2))
        else:
            angles[f"{aType} - {bType} - {cType}"] = [(bond_angle, charge[angle[0]], charge[angle[1]], charge[angle[2]], bond_order1, bond_order2)]                                                                                      
                                                                                                     
    return angles

In [23]:
def getTorsions(mol, bondOrder, charge) -> dict:
    '''Iterate through all the torsions in a molecule'''
    
    torsions = {}
    
    for torsion in ob.OBMolTorsionIter(mol.OBMol):
        a = torsion[0] + 1
        b = torsion[1] + 1
        c = torsion[2] + 1
        d = torsion[3] + 1
        
        bondOrder1 = bondOrder[torsion[0], torsion[1]]
        bondOrder2 = bondOrder[torsion[1], torsion[2]]
        bondOrder3 = bondOrder[torsion[2], torsion[3]]
        
        torsion_angle = round(mol.OBMol.GetTorsion(a, b, c, d), 3)

        aType = filterAtom(atomType(mol, a))
        bType = filterAtom(atomType(mol, b))
        cType = filterAtom(atomType(mol, c))
        dType = filterAtom(atomType(mol, d))

        # Switch if not in lexographic order
        if(dType < aType):
            aType, dType = dType, aType
            bondOrder1, bondOrder3 = bondOrder3, bondOrder1
            
        #Appedning to the dictonary of lists if the torsion angle b/t the elements already existse
        if(f"{aType} - {bType} - {cType} -{dType}" in torsions.keys()):
            torsions[f"{aType} - {bType} - {cType} -{dType}"].append((torsion_angle, charge[torsion[0]], charge[torsion[1]],
                                                charge[torsion[2]], charge[torsion[3]], bondOrder1, bondOrder2, bondOrder3))
        
        # Checking for palindromic sequence
        elif(f"{dType} - {cType} - {bType} -{aType}" in torsions.keys()):
            torsions[f"{dType} - {cType} - {bType} -{aType}"].append((torsion_angle, charge[torsion[3]], charge[torsion[2]],
                                                charge[torsion[1]], charge[torsion[0]], bondOrder3, bondOrder2, bondOrder1))
            
        else:
            torsions[f"{aType} - {bType} - {cType} -{dType}"] = [(torsion_angle, charge[torsion[0]], charge[torsion[1]],
                                            charge[torsion[2]], charge[torsion[3]], bondOrder1, bondOrder2, bondOrder3)]
    
    return torsions

### Wrapping all the helper functions into a main function

In [28]:
def analyze_molecular_data(file: str) -> (dict, dict, dict):
    '''Fetches the bond lengths, angles and dihedrals from a given file'''
    
    # Load the File
    extension=file.split('.')[-1]
    try:
        mol = next(pybel.readfile(extension, file))
    except:
        error_files.append(file)
        return ({},{},{})
    
    # Get Bond Order and partial charges on atoms
    charge, bond_order = getBondOrders_Charges(mol)
    
    # Get bond lengths
    #bond_lengths = getBonds(mol, charge, bond_order)
    
    # Get bond angles
    bond_angles = getAngles(mol, bond_order, charge)
    
    # Get torsions
    dihedrals = getTorsions(mol, bond_order, charge)

    return (bond_lengths, bond_angles, dihedrals)


def mergeDictionaries(dict1 : dict, dict2: dict):
    '''Merges two Python dictionary by combining elements for a common key
    The function merges dict2 into dict 1'''
    for key, value in dict2.items():
        if key in dict1:
            dict1[key].extend(value)
        else:
            dict1[key] = value

### Testing the above functions on a sample molecule

In [24]:
# Example usage
xyz_file = '21395061/cod-crest/W/WKGCLMYDEAXJHN-UHFFFAOYSA-N.xyz'
mol = next(pybel.readfile('xyz', xyz_file))
charge, bond_order = getBondOrders_Charges(mol)
#pprint(getBonds(mol))
#getAngles(mol, bond_order, charge)
getTorsions(mol, bond_order, charge)


#molecular_data = analyze_molecular_data(xyz_file)
#molecular_data

{'H - C - C -H': [(-61.147,
   0.03503899968442077,
   -0.10378630636174588,
   -0.053875656833530024,
   0.03290577940117682,
   0.987853806853633,
   1.0192875179900571,
   0.9784312829508657),
  (-178.881,
   0.03503899968442077,
   -0.10378630636174588,
   -0.053875656833530024,
   0.032890953517818074,
   0.987853806853633,
   1.0192875179900571,
   0.9784238569929431),
  (58.925,
   0.036001701940108675,
   -0.10378630636174588,
   -0.053875656833530024,
   0.03290577940117682,
   0.9872833720193072,
   1.0192875179900571,
   0.9784312829508657),
  (-58.81,
   0.036001701940108675,
   -0.10378630636174588,
   -0.053875656833530024,
   0.032890953517818074,
   0.9872833720193072,
   1.0192875179900571,
   0.9784238569929431),
  (178.989,
   0.03503443734946105,
   -0.10378630636174588,
   -0.053875656833530024,
   0.03290577940117682,
   0.9878446566377561,
   1.0192875179900571,
   0.9784312829508657),
  (61.255,
   0.03503443734946105,
   -0.10378630636174588,
   -0.053875656833

## Extracting Molecular Data from all the files

In [29]:
#Collecting data from the Molecular Dataset

bond_lengths = {}
bond_angles = {}
dihedrals = {}

ext = ('.xyz',)
count = 0

for root, dirs, files in os.walk('21395061/cod-crest/'):
    for file in files:
        extension=os.path.splitext(file)[1]
        count +=1
        if(extension not in ext): continue
            
        filename = os.path.join(root,file)
        length, angle, torsion = analyze_molecular_data(filename)
        
        #mergeDictionaries(bond_lengths, length)
        mergeDictionaries(bond_angles, angle)
        mergeDictionaries(dihedrals, torsion)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -59.433677868596          0.000316298731)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -79.377053843214          0.000924825445)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -86.034286802623          0.000854184241)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -58.758444780557          0.000328546459)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done       -111.085364902439          0.000603406375)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::Perceive

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -94.340217728248          0.000410361877)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -65.485149618314          0.000041819052)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -84.893951507837          0.000432179138)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -43.338442248229          0.000158257975)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done       -161.206963397959          0.000759112675)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::Perceive

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done       -122.706261229453          0.000619626115)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -45.800808954851          0.000127003290)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done       -116.316247123820          0.000341302592)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -72.235098405002          0.000574729245)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -33.667170665269          0.000397739222)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::Perceive

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -82.258093546117          0.000946077429)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done       -130.114727446468          0.000453489848)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done       -126.237724477994          0.000918757918)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -51.742702166545          0.000363859104)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -31.789058681207          0.000159162384)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::Perceive

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -48.498179052821          0.000647271907)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -86.850574119305          0.000660827113)

*** Open Babel Warning  in ReadMolecule
  Problems reading an XYZ file: Cannot read the first line.
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -98.944952818855          0.000303391322)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -72.047500907270          0.000736772944)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -62.761696921750          0.000756825227)



...

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -59.804318824484          0.000110920937)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -60.376897728145          0.000300592719)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -53.051967056058          0.000319347612)

*** Open Babel Warning  in ReadMolecule
  Problems reading an XYZ file: Cannot read the first line.
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -98.442687653706          0.000729921947)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -55.024029917737          0.000846533144)



*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -62.640663187558          0.000075419240)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is -51.73371118)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is -126.81155588)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -28.756519489538          0.000058683699)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -68.467199307241          0.000035712067)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is -157.33012532)

*** Open Babel Warning  in PerceiveBondOrders
  Fa

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is -150.19221434)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is -92.87677807)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done       -171.343066796213          0.000018589280)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -76.526232883783          0.000039041893)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -41.752312203978          0.000065903802)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is -58.29767816)

*** Open Babel Warning  in PerceiveBondOrders
  Fai

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is -57.71487871)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is -133.48170259)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is -138.56009644)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -46.564518915949          0.000123821791)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is -107.43700611)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -36.961859227693          0.000024709691)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::P

XTBException: Single point calculation failed:
-2- xtb_calculator_singlepoint: Electronic structure method terminated
-1- scf: Self consistent charge iterator did not converge

### Running the corrupt xyz files with corresponding sdf files and writing the bond length, angle, torsion data as pickle object

In [30]:
#Trying with sfd files for the corrupted xyz files

for file in error_files:
    filename = os.path.splitext(file)[0] + '.sdf'
    length, angle, torsion = analyze_molecular_data(filename)
    #mergeDictionaries(bond_lengths, length)
    mergeDictionaries(bond_angles, angle)
    mergeDictionaries(dihedrals, torsion)

In [ ]:
def MolecularDataWriter_compress():
    
    pickled_bond_lengths = pickle.dump(bond_lengths)
    compressed_bond_lengths = blosc.compress(pickled_bond_lengths)
    
    with open('cod-crest_bond_lengths.dat', 'wb') as f:
        f.write(compressed_bond_lengths)
        
    pickled_bond_angles = pickle.dump(bond_angles)
    compressed_bond_angles = blosc.compress(pickled_bond_angles)
        
    with open('cod-crest_bond_angles.dat', 'wb') as f:
        f.write(compressed_bond_angles)
        
    pickled_torsions = pickle.dump(torsion)
    compressed_dihedrals = blosc.compress(pickled_torsions)
        
    with open('cod-crest_torsions.dat', 'wb') as f:
        f.write(compressed_dihedrals)

In [31]:
def MolecularDataWriter_pickle():
    
    for key in bond_lengths.keys():
        bond_lengths[key] = np.array(bond_lengths[key])
        
    for key in bond_angles.keys():
        bond_angles[key] = np.array(bond_angles[key])
        
    for key in dihedrals.keys():
        dihedrals[key] = np.array(dihedrals[key])
    
#     with open('cod-crest_bond_lengths_.pkl', 'wb') as f:
#         pickle.dump(bond_lengths, f)
        
    with open('cod-crest_bond_angles.pkl', 'wb') as f:
        pickle.dump(bond_angles, f)
        
    with open('cod-crest_torsions.pkl', 'wb') as f:
        pickle.dump(dihedrals, f)

In [34]:
for key in bond_angles.keys():
    print(key)

Car - O - Nar
Car - Nar - Car
Car - O - Car
Nar - Car - Nar
Car - Car - Nar
Car - Car - Car
Car - H - Car
Car - Car - H
C - Nar - Nar
C - Car - Nar
Car - Nar - Nar
C - Nar - Car
C - Car - Car
Car - Car - Npl
H - Car - Npl
H - H - Npl
C - Car - H
C - H - H
C - Nar - H
C - O - C
C - C - C
C - C - H
C - O - O
C - C - O
C - O - H
Car - F - Car
C - Car - Nam
C - O - Nam
C - Car - O
Car - H - Nam
H - C - Nam
Car - C - Nam
Car - Nam - Car
C - F - F
C - Car - F
Car - O - Ntr
Ntr - O - O
Car - Ntr - Car
Car - C - Car
C - F - Ntr
C - Ntr - C
C - F - C
Ntr - C - O
C - C - Car
Car - Car - O
C - O - Ntr
C - Ntr - H
C - C - N
C - N - H
N - C - O
HO - N - O
C - O - Car
Car - Car - S
Car - S - Nar
Car - S - Car
B - C - N
C - N - C
B - N - Car
B - C - Car
B - N - C
C - B - N
B - C - C
C - B - H
Car - B - Car
C - Car - C
Car - Cl - Car
Car - Car - Cl
C - Car - N
C - N - O
C - O - N
C - N - Car
C - Npl - N
C - N - Npl
C - Npl - Car
C - Npl - C
N - C - Npl
C - Nam - C
C - C - Nam
C - Nam - H
C - Nam - O
C

In [32]:
MolecularDataWriter_pickle()